In [1]:
import scanpy
import pandas as pd

## Clean Up Variable Names (X axis columns)
#### Uses a dictionary to rename selective columns and resaves the data as a 10x h5 file

In [2]:
gene_dict = {"gene_orf1ab": "COVID-orf1ab", "gene_S": "COVID-S", "gene_ORF3a": "COVID-ORF3a", "gene_E": "COVID-E", "gene_M": "COVID-M",
             "gene_ORF6": "COVID-ORF6", "gene_ORF7a": "COVID-ORF7a", "gene_ORF8": "COVID-ORF8", "gene_N":"COVID-N", "gene_ORF10": "COVID-ORF10"}

file_name = "../data/GSM4674665_CS9_CS10_raw_feature_bc_matrix.h5"
adata = scanpy.read_10x_h5(file_name)

print(adata.var_names)

new_var_names = []
for item in adata.var_names:
    if "gene_" in item:
        new_var_names.append(gene_dict[item])
    else:
        new_var_names.append(item)

adata.var_names = new_var_names

print(adata.var_names)

/home/mdix/.conda/envs/10x_covid_py/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


Index(['RP11-34P13.3', 'FAM138A', 'OR4F5', 'RP11-34P13.7', 'RP11-34P13.8',
       'RP11-34P13.14', 'RP11-34P13.9', 'FO538757.3', 'FO538757.2',
       'AP006222.2',
       ...
       'gene_orf1ab', 'gene_S', 'gene_ORF3a', 'gene_E', 'gene_M', 'gene_ORF6',
       'gene_ORF7a', 'gene_ORF8', 'gene_N', 'gene_ORF10'],
      dtype='object', length=33704)
Index(['RP11-34P13.3', 'FAM138A', 'OR4F5', 'RP11-34P13.7', 'RP11-34P13.8',
       'RP11-34P13.14', 'RP11-34P13.9', 'FO538757.3', 'FO538757.2',
       'AP006222.2',
       ...
       'COVID-orf1ab', 'COVID-S', 'COVID-ORF3a', 'COVID-E', 'COVID-M',
       'COVID-ORF6', 'COVID-ORF7a', 'COVID-ORF8', 'COVID-N', 'COVID-ORF10'],
      dtype='object', length=33704)


/home/mdix/.conda/envs/10x_covid_py/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [12]:
import h5py
import numpy as np
from scipy.sparse import csr_matrix
from pathlib import Path

# Scanpy does not have a native write 10x h5 function
# This version is copied from here: https://github.com/scverse/anndata/issues/595
def write_10X_h5(adata, file):
    """Writes adata to a 10X-formatted h5 file.
    
    Note that this function is not fully tested and may not work for all cases.
    It will not write the following keys to the h5 file compared to 10X:
    '_all_tag_keys', 'pattern', 'read', 'sequence'

    Args:
        adata (AnnData object): AnnData object to be written.
        file (str): File name to be written to. If no extension is given, '.h5' is appended.

    Raises:
        FileExistsError: If file already exists.

    Returns:
        None
    """
    
    if '.h5' not in file: file = f'{file}.h5'
    if Path(file).exists():
        raise FileExistsError(f"There already is a file `{file}`.")
    def int_max(x):
        return int(max(np.floor(len(str(int(max(x)))) / 4), 1) * 4)
    def str_max(x):
        return max([len(i) for i in x])

    w = h5py.File(file, 'w')
    grp = w.create_group("matrix")
    grp.create_dataset("barcodes", data=np.array(adata.obs_names, dtype=f'|S{str_max(adata.obs_names)}'))
    grp.create_dataset("data", data=np.array(adata.X.data, dtype=f'<i{int_max(adata.X.data)}'))
    ftrs = grp.create_group("features")
    # this group will lack the following keys:
    # '_all_tag_keys', 'feature_type', 'genome', 'id', 'name', 'pattern', 'read', 'sequence'
    ftrs.create_dataset("feature_type", data=np.array(adata.var.feature_types, dtype=f'|S{str_max(adata.var.feature_types)}'))
    ftrs.create_dataset("genome", data=np.array(adata.var.genome, dtype=f'|S{str_max(adata.var.genome)}'))
    ftrs.create_dataset("id", data=np.array(adata.var.gene_ids, dtype=f'|S{str_max(adata.var.gene_ids)}'))
    ftrs.create_dataset("name", data=np.array(adata.var.index, dtype=f'|S{str_max(adata.var.index)}'))
    grp.create_dataset("indices", data=np.array(adata.X.indices, dtype=f'<i{int_max(adata.X.indices)}'))
    grp.create_dataset("indptr", data=np.array(adata.X.indptr, dtype=f'<i{int_max(adata.X.indptr)}'))
    grp.create_dataset("shape", data=np.array(list(adata.X.shape)[::-1], dtype=f'<i{int_max(adata.X.shape)}'))


print(file_name[:-3])
write_10X_h5(adata, file_name[:-3]+"_clean.h5")

test
../data/GSM4674665_CS9_CS10_raw_feature_bc_matrix
